In [22]:
import math
import random
import matplotlib.pyplot as plt
import tqdm

class DBSCAN(object):

    STATUS_UNVISITED = 'unvisited'
    STATUS_VISITED = 'visited'

    STATUS_GROUP = 1
    STATUS_NOGROUP = 0

    data = dict()

    def __init__(self, e, minPts):
        """
        e 最小距离
        minPts 最少样本数量
        """
        self.e = e
        self.minPts = minPts

    def nearby(self, id):
        nearby_points = list()
        for link_id in range(len(self.scores[id])):
            if self.scores[id][link_id] <= self.e:
                nearby_points.append(link_id)

        return nearby_points

    def visit_nearby_points(self, points, group):
        for id in points:
            if self.data[id]['is_visited'] == self.STATUS_VISITED \
                    and self.data[id]['is_group'] == self.STATUS_GROUP:
                continue
            self.data[id]['is_visited'] = self.STATUS_VISITED

            if self.data[id]['is_group'] == self.STATUS_NOGROUP:
                group.append(id)
                self.data[id]['is_group'] = self.STATUS_GROUP

            nearby_points = self.nearby(id)
            if len(nearby_points) >= self.minPts:
                self.visit_nearby_points(nearby_points, group)

    def fit(self, data_set, scores):
        self.scores = scores
        groups = list()

        for index, item in enumerate(data_set):
           self.data[index] = {'id': index,
                                'is_visited': self.STATUS_UNVISITED,
                                'is_group': self.STATUS_NOGROUP
                                }
        print(index)

        for id in self.data:
            if self.data[id]['is_visited'] == self.STATUS_VISITED:
                continue

            self.data[id]['is_visited'] = self.STATUS_VISITED
            nearby_points = self.nearby(id)

            if len(nearby_points) >= self.minPts:
                group = list()
                group.append(id)
                self.data[id]['is_group'] = self.STATUS_GROUP
                self.visit_nearby_points(nearby_points, group)
                groups.append(group)

        for id in self.data:
            if self.data[id]['is_group'] == self.STATUS_NOGROUP:
                groups.append([id])

        return groups


def init_data(num, min, max):
    data = []
    for i in range(num):
        data.append([random.randint(min, max), random.randint(min, max)])

    return data


def mat_score(data_set):
    score = dict()
    for i in range(len(data_set)):
        score[i] = dict()

    for i in range(len(data_set) - 1):
        j = i + 1
        while j < len(data_set):
            score[i][j] = math.sqrt(abs(data_set[i][0] - data_set[j][0]) ** 2 + abs(data_set[i][1] - data_set[j][1]) ** 2)
            score[j][i] = score[i][j]
            j += 1

    return score


# def show_cluster(data_set, groups):
#     plt.title(u'DBSCAN')
#     mark = ['or', 'ob', 'og', 'ok', '^r', '+r', 'sr', 'dr', '<r', 'pr']
#     for index, group in enumerate(groups):
#         for i in group:
#             plt.plot(data_set[i][0], data_set[i][1], mark[index])

#     plt.xlim(0.0, 100)
#     plt.ylim(0.0, 100)
#     plt.show()



In [4]:
import numpy as np
 

#导入npy文件路径位置
test = np.load('C:\\Users\\15957\\Desktop\\2023\\data mining\\project\\npy_embs\\ift_cluster_given_fudandm2023-input-bert-base-chinese.npy')

print(test.shape)

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

print(f">>> t-SNE fitting")
tsne = TSNE(n_components=2, init='pca', perplexity=30)
Y = tsne.fit_transform(test)
print(f"<<< fitting over")
print(Y.shape)


(33715, 768)
>>> t-SNE fitting


C:\Users\15957\AppData\Roaming\Python\Python38\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
C:\Users\15957\AppData\Roaming\Python\Python38\site-packages\sklearn\manifold\_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<<< fitting over
(33715, 2)


In [8]:
print(Y[0])
np.save("2demb",Y)

[-13.63955   -6.073564]


In [ ]:
dists = []
for i in range(20):
    dist = []
    for j in range(20):
        dist.append(np.linalg.norm(np.array(Y[i]) - np.array(Y[j])))
    dists.append(dist)
   
print(dists)


In [12]:
# scores = np.array(dists)
# print(scores[0])

MemoryError: Unable to allocate 4.23 GiB for an array with shape (33715, 33715) and data type float32

In [4]:
import numpy as np
Y = np.load("2demb.npy")

In [ ]:
with open("dists.txt","w") as f:
    for i in range(len(Y) - 1):
        j = 0
        if i%100 == 0:
            print(i)
        while j < len(Y):
            dist = np.linalg.norm(np.array(Y[i]) - np.array(Y[j]))
            f.write(str(dist))
            f.write(" ")
            j += 1
        f.write("\n")
    f.close()

In [32]:
data = []
for i in range(20):
    data.append(Y[i])
model = DBSCAN(50,50)
data = np.array(data)
dists = np.array(dists)
print(data.shape)
print(dists.shape)
cluster = model.fit(data, dists)
print(cluster)
np.save('dbscan', cluster)


(20, 2)
(20, 20)
19
[[0, 2, 3, 4, 5, 6, 1, 7, 9, 8, 10, 11, 12, 13, 15, 14, 18, 16, 19, 17]]
